In [1]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

In [19]:
# Run for interactive plots with jupyter lab
%matplotlib widget          

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy import stats
import statsmodels.api as sm

import matplotlib
# matplotlib.use('nbagg')    # Doesn't work with jupyter lab
import matplotlib.pyplot as plt


# Ordinary Least Squares

In multiple regression, we have a model of the form
\begin{equation}
\mathbf{y = X \boldsymbol{\beta + \epsilon}} \quad \boldsymbol{\epsilon} \sim N(\mathbf{0},\sigma\mathbf{I})
\end{equation}

where $\mathbf{y}$ is an $n\times 1$ vector, $X$ is an $n \times k$ design matrix, $\boldsymbol{\beta}$ is the vector of $k$ coefficients, and $\boldsymbol{\epsilon}$ is the $n \times 1$ vector of standard normal random errors.

We derive an estimate $\mathbf{b}$ of $\boldsymbol{\beta}$ from a sample of observations for which
\begin{equation}
\label{eq-Reg}
\mathbf{y = Xb + e}
\end{equation}
by minimizing the squared sum of errors $\mathbf{e'e}$. The estimate is given by
\begin{equation}
\label{eq-b}
\mathbf{b = \left(X'X\right)^{-1} X'y}
\end{equation}

Plugging equation (\ref{eq-b}) into equation (\ref{eq-Reg}), we get
\begin{equation}
\label{eq-H}
\mathbf{y = X \left(X'X\right)^{-1} X'y + e \equiv Hy + e }
\end{equation}

The predicted values $\mathbf{Hy}$ are often denoted $\mathbf{\hat{y}}$, which explains the name "hat matrix" for $\mathbf{H}$.

In [16]:
# Creating artificial data for regression

np.random.seed(1)       # For reproducibility
X = np.random.random((30,2))
b = [0.2,0.3]
e = np.random.random(30)

y = np.dot(X,b) + e

stats.describe(e)
# Running the regression

X = sm.add_constant(X)        # Adding an intercept to the regression

m1 = sm.OLS(y,X).fit()
print(m1.summary())

DescribeResult(nobs=30, minmax=(0.04995345894608716, 0.9648400471483856), mean=0.5558004906644833, variance=0.0770657116798012, skewness=-0.23634961375889468, kurtosis=-1.0681272762337057)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.182
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:                     3.002
Date:                Fri, 22 May 2020   Prob (F-statistic):             0.0665
Time:                        16:25:24   Log-Likelihood:                -3.4473
No. Observations:                  30   AIC:                             12.89
Df Residuals:                      27   BIC:                             17.10
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5371      0.120      4.490      0.0

## Diagnostics

### R squared, F stat

The regression has $n$ observations, $\mathbf{X}$ is an $n\times k$ matrix that has $p=k-1$ predictors and includes an intercept.

**R squared and adjusted R squared**

Regression can be analyzed in terms of an attribution of the sum of squared deviations of the dependent or response variable $\sum_i (y_i-\bar{y})^2$. Fot this, since $y_i = \hat{y}_i + e_i$, we note that
$$
\sum_i (y_i-\bar{y})^2 = \sum_i (\hat{y}_i - \bar{y} +e_i)^2 = \sum_i (\hat{y}_i-\bar{y})^2 + \sum_i e_i^2 
$$
since $\sum_i \hat{y}_i e_i = 0$ and $\sum_i e_i = 0$.

This is often written as
$$
\text{Total sum of squares (SST)} = \text{Regression sum of squares (SSR)} + \text{Error sum of squares (SSE)}
$$

The coefficient of determination, $R^2$ is defined as
$$
R^2 = \frac{SSR}{SST} = 1 - \frac{SSE}{SST}
$$

$R^2$ gives us the fraction of the total variation in $y$ accounted for by the regression and is used as a measure of the goodness of fit for the model. One of its drawbacks is that it always increases with the addition of a new variable, because SSE decreases [For a proof, see Greene, William H., *Econometric Analysis*,4th edition, p. 235]. For this reason, an adjusted value is defined:

$$
R^2_{adj} = 1 - \frac{SSE/(n-k)}{SST/(n-1)} = 1 - \frac{n-1}{n-k}\;(1-R^2)
$$


**The F statistic in multiple regression**

 The F statistic is
$$
F = \frac{\frac{SSR}{k-1}}{\frac{SSE}{[n-k]}} = \frac{MSR}{MSE}
$$

where $SSR$ is the regression sum of squares, $SSE$ is the sum of squared errors, and $MSR$ and $MSE$ are their means respectively.

Since $R^2 = \frac{SSR}{SST}$,
$$
\begin{split}
F &= \frac{SSR}{SSE} \frac{[n-k]}{k-1}\\
  & = \frac{R^2}{1-R^2}\frac{[n-k]}{k-1}
\end{split}
$$

**Degrees of freedom**

|Source | Degrees of Freedom |
|---        |---   |
| Regression | $p=k-1$ |
| Error      | $n-k$   |
| Total      | $n-1$   |


In [90]:
# Sums of squares, R^2, adjusted R^2, and F stat for overall model fit

# SSTotal = SSReg + SSError ;   R^2 = SSReg/SSTotal ; F = (SSReg/df_reg)/(SSError/df_resid) 

# statsmodels uses confusing variable names
# SSReg --> ess   and SSError --> ssr
# centered_tss = ess (explained sum of squares) + ssr (sum of squared residuals)

print("SST = ",m1.centered_tss, "SSR = ", m1.ess, "SSE = ", m1.ssr, "SSR + SSE = ", m1.ess+m1.ssr) 

#m1.df_model, m1.df_resid         # degrees of freedom in case we need them

# Confirm rsquared calculations

r2 = m1.ess/m1.centered_tss
print("\nR-squared check: ",r2,m1.rsquared)  

r2a = 1-(m1.ssr/m1.df_resid)/(m1.centered_tss/(m1.df_model+m1.df_resid))
print("\nAdjusted R-squared check: ",r2a,m1.rsquared_adj)

# Confirm F statistic calculation
F = m1.mse_model/m1.mse_resid
Fpvalue = 1-stats.f.cdf(F,m1.df_model,m1.df_resid)

print("\nF stat check: ",F, m1.fvalue   ) 
print("\nProb(F) check: ",Fpvalue, m1.f_pvalue)

SST =  2.7018948326204675 SSR =  0.4915620365070974 SSE =  2.21033279611337 SSR + SSE =  2.7018948326204675

R-squared check:  0.18193233525316366 0.18193233525316366

Adjusted R-squared check:  0.12133473045710164 0.12133473045710164

F stat check:  3.0023024155071365 3.0023024155071365

Prob(F) check:  0.06647442100717305 0.0664744210071729


### Model selection: Log likelihood, AIC, BIC

### Normality of residuals: Jarque-Bera test

[Wikipedia entry](https://en.wikipedia.org/wiki/Jarque%E2%80%93Bera_test).

$$
JB ={\frac {n}{6}}\left(S^{2}+{\frac {1}{4}}(K-3)^{2}\right)
$$
where
$$
\begin{split}
S &= {\frac{{\hat{\mu }}_{3}}{{\hat  {\sigma }}^{3}}}={\frac  {{\frac  1n}\sum _{{i=1}}^{n}(x_{i}-{\bar  {x}})^{3}}{\left({\frac  1n}\sum _{{i=1}}^{n}(x_{i}-{\bar  {x}})^{2}\right)^{{3/2}}}}\\
K &= {\frac{{\hat{\mu }}_{4}}{{\hat  {\sigma }}^{4}}} ={\frac  {{\frac  1n}\sum _{{i=1}}^{n}(x_{i}-{\bar  {x}})^{4}}{\left({\frac  1n}\sum _{{i=1}}^{n}(x_{i}-{\bar  {x}})^{2}\right)^{{2}}}}
\end{split}
$$

The test statistic $JB$ is approximately $\chi^2$ distributed with 2 degrees of freedom for large $n$.

In [86]:
# JB statistic for regression residuals
x = m1.resid
n = np.shape(x)[0]

xdev = x - np.mean(x)
mvec = np.zeros(3)
for i in range(len(mvec)):
    mvec[i] = np.sum(xdev**(i+2))/n
    
M = dict(zip(['sigma2','mu3','mu4'],mvec))
         
S = M['mu3']/(M['sigma2']**(3/2))
K = M['mu4']/(M['sigma2']**2)

JB = (n/6)*(S**2 + (1/4)*(K-3)**2)   # H0: S=0, K-3=0 (sample data are from normal distribution)

print("Skewness = ",S,"Kurtosis = ",K,"Test Statistic = ",JB)
print("p value = ",1-stats.chi2.cdf(JB,2))   # p value = Prob (X >= JB) in chi-squared distribution with dof=2
                                             # Large p-value indicates that we fail to reject the null hypothesis

Skewness =  -0.31020796231133496 Kurtosis =  1.9180548173303456 Test Statistic =  1.944401622284343
p value =  0.378249665313768


### Autocorrelation of residuals: Durbin-Watson test

### Residual plots

In [62]:
# Residual vs. predicted value

fig = plt.figure()
ax = fig.add_subplot(111,title="Residual plot",ylabel=r"$e_i$",xlabel=r"$\haty$")

y = m1.resid
x = m1.predict()
_=plt.scatter(x,y)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### The Omnibus Test

This test is for the normality of the distribution of residuals. [See Wikipedia entry](https://en.wikipedia.org/wiki/D%27Agostino%27s_K-squared_test).

In [66]:
# Omnibus test diagnostic reported by summary
stats.normaltest(m1.resid)

NormaltestResult(statistic=4.062075177125114, pvalue=0.13119931958670275)

In [76]:
help(sm.OLS.fit)
help(sm.regression.linear_model.RegressionResults)

Help on function fit in module statsmodels.regression.linear_model:

fit(self, method='pinv', cov_type='nonrobust', cov_kwds=None, use_t=None, **kwargs)
    Full fit of the model.
    
    The results include an estimate of covariance matrix, (whitened)
    residuals and an estimate of scale.
    
    Parameters
    ----------
    method : str, optional
        Can be "pinv", "qr".  "pinv" uses the Moore-Penrose pseudoinverse
        to solve the least squares problem. "qr" uses the QR
        factorization.
    cov_type : str, optional
        See `regression.linear_model.RegressionResults` for a description
        of the available covariance estimators.
    cov_kwds : list or None, optional
        See `linear_model.RegressionResults.get_robustcov_results` for a
        description required keywords for alternative covariance
        estimators.
    use_t : bool, optional
        Flag indicating to use the Student's t distribution when computing
        p-values.  Default behavior d